In [60]:
import streamlit as st
from streamlit import session_state as ss
from langchain_core.runnables import RunnableParallel

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_pinecone import PineconeVectorStore
from pymongo import MongoClient
import os

load_dotenv()

def vector_db():
    index_name = os.getenv("PINECONE_INDEX_NAME")
    embeddings_size = 3072
    embeddings_model = 'text-embedding-3-large'
    embeddings = OpenAIEmbeddings(model=embeddings_model, dimensions=embeddings_size)
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

def vector_db_simu():
    MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_URI')
    client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
    DB_NAME = os.getenv("DB_NAME")
    COLLECTION_NAME = os.getenv("COLLECTION_NAME")
    embeddings_model = 'text-embedding-3-large'
    ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv("ATLAS_VECTOR_SEARCH_INDEX_NAME")
    MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
    vectorstore = MongoDBAtlasVectorSearch(
        embedding=OpenAIEmbeddings(model=embeddings_model,
                                   dimensions=1536),
        collection=MONGODB_COLLECTION,
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    )
    return vectorstore

In [61]:
vdb_simu = vector_db_simu()
vdb = vector_db()

In [62]:
retriever_simu = vdb_simu.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever = vdb.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [63]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

def format_questoes(questoes):
    return "\n\n".join([tema for tema in questoes])

In [ ]:
def get_chain_mock(model="openai"):
    
    if model == "openai":
        _chat_ = ChatOpenAI
        _model_name_ = "gpt-4o-2024-08-06"
    elif model == "groq":
        _chat_ = ChatGroq
        _model_name_ = "llama-3.2-90b-text-preview"
    
    template = """
    Você é um assistente de modelo de linguagem de IA que irá gerar simulados. Sua tarefa é separar cada uma das matérias ou temas
    que o usuário deseja estudar e gerar uma curta query descrevendo o temas, para facilitar a busca por similaridade.
    A entrada do usuário pode ter uma ou mais matérias ou temas.
    Forneça esses temas/matérias separados por novas linhas, no formato:
    - Tema1: Descricao do tema1
    - Tema2: Descricao do tema2
    e assim por diante.
    Separe os temas em: {user_query}"""
    
    template_1 = [
        ('system', """
                    Você é um bot chamado Edu e foi desenvolvido pela Nero.AI. 
                    Você irá gerar questões para compor um simulado do ENEM."""),
        ('system', "Use essas questões como exemplo de estrutura de questão\n\n{context_query_simu}\n\n"),
        ('system', "Use esses documentos para embasar o conteúdo das questões\n\n{context_query_video}\n\n"),
        ('system', "Não crie questões unicamente objetivas, como 'O que é?', 'Quem foi?', contextualize breventemente o tema e crie questões que exijam raciocínio."),
        ('system', """Forneça essas perguntas alternativas separadas por novas linhas. \n:
            Siga a estrutura usando markdown:
            Tema: (tema_questao)\n
         
            QUESTÃO 1: (questao_1)\n
            a) alternativa 1
            b) alternativa 2
            c) alternativa 3
            d) alternativa 4
            e) alternativa 5

            QUESTÃO 2: (questao_2)
            a) alternativa 1
            b) alternativa 2
            c) alternativa 3
            d) alternativa 4
            e) alternativa 5

        e assim por diante.
            Gabarito: 1 - (letra correspondente a alternativa)\n
                      2 - (letra correspondente a alternativa)\n
        e assim por diante.
        """),
        ('system', "Gere apenas 5 questões sobre o tema: {tema_questao}"),
    ]
    
    prompt1 = ChatPromptTemplate.from_messages(template_1)
    
    prompt = ChatPromptTemplate.from_template(template)

    llm = _chat_(temperature=0.1, model=_model_name_)

    generate_temas = (
        prompt
        | llm
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    )

    chain_rag_simu = retriever_simu | format_docs
    chain_rag_video = retriever | format_docs

    chain_rag = (
        {
            "context_query_simu": RunnablePassthrough() | chain_rag_simu,
            "context_query_video": RunnablePassthrough() | chain_rag_video,
            "tema_questao": RunnablePassthrough()
        }
        | prompt1
        | llm
        | StrOutputParser()
    )
    
    chain = generate_temas | chain_rag.map() | format_questoes | StrOutputParser()
    
    return chain

In [65]:
chain = get_chain_mock(model="groq")

In [66]:
response_simulado = chain.invoke({"user_query": user_query})

In [67]:
print(response_simulado)

Tema: História do Brasil: Estudo da evolução política, social e econômica do Brasil desde a colonização portuguesa até os dias atuais.

QUESTÃO 1: Qual foi o principal motivo para a colonização portuguesa no Brasil durante o século XVI?

a) A busca por ouro e prata
b) A expansão do cristianismo
c) A exploração da cana-de-açúcar
d) A competição com os holandeses e franceses
e) A necessidade de estabelecer uma rota comercial com a Ásia

QUESTÃO 2: Como a colonização portuguesa afetou a população indígena no Brasil?

a) A população indígena foi completamente exterminada
b) A população indígena foi integrada à sociedade portuguesa
c) A população indígena foi forçada a trabalhar em condições de escravidão
d) A população indígena foi isolada em reservas
e) A população indígena foi protegida pelos portugueses

QUESTÃO 3: Qual foi o papel dos africanos na colonização portuguesa do Brasil?

a) Eles foram trazidos como colonos
b) Eles foram trazidos como escravos para trabalhar nas plantações de

## Retriever

In [68]:
chain_rag_simu = retriever_simu 
chain_rag_video = retriever

In [69]:
response_simu = chain_rag_simu.invoke(" Qual é o significado do termo eurocentrismo no contexto da história do Brasil?")

In [70]:
[x.page_content for x in response_simu]

['Questão 8          \nSlam do Corpo é um encontro pensado para surdos \ne ouvintes, existente desde 2014, em São Paulo. Uma \niniciativa pioneira do grupo Corposinalizante, criado em \n2008. (Antes de seguirmos, vale a explicação: o termo \nslam vem do inglês e significa — numa nova acepção \npara o verbo geralmente utilizado para dizer “bater com \nforça” — a “poesia falada nos ritmos das palavras e da \ncidade”). Nos saraus, o primeiro objetivo foi o de botar \nos poemas em Libras na roda, colocar os surdos para \ncircular e entender esse encontro entre a poesia e a língua \nde sinais, compreender o encontro dessas duas línguas. \nPoemas de autoria própria, três minutos, um microfone. \nSem figurino, nem adereços, nem acompanhamento \nmusical. O que vale é modular a voz e o corpo, um \ntrabalho artesanal de tornar a palavra “visível”, numa \narena cujo objetivo maior é o de emocionar a plateia, tirar \no público da passividade, seja pelo humor, horror, caos, \ndoçura e outras tantas

In [71]:
print(chain_rag_video.invoke("Historia do brasil", k=3))

[Document(id='6ffc83d1-548f-4466-aeb4-cfd29d803730', metadata={'author': 'Parabólica', 'description': 'Unknown', 'length': 811.0, 'publish_date': '2018-04-27 00:00:00', 'source': '5Khp_i66eP8', 'thumbnail_url': 'https://i.ytimg.com/vi/5Khp_i66eP8/hq720.jpg', 'title': 'APRENDENDO A HISTÓRIA DO SÉCULO XVI.', 'view_count': 48101.0}, page_content='Title: APRENDENDO A HISTÓRIA DO SÉCULO XVI.\nViews count: 48101\nPublish date: 2018-04-27 00:00:00\nAuthor: Parabólica\nDescription: Unknown\nLink video: https://www.youtube.com/watch?v=5Khp_i66eP8\n\nem luta contra os protestantes então o brasil começa a receber e que são fundadas por exemplo cidades importantes que não eram cidades eram vilas mas cuidado salvador que são salvador rio de janeiro que foi fundado para poder expulsar os franceses a são paulo são paulo de piratininga tão brasil está recebendo uma estrutura administrativa a gente começa a ter aqui no brasil há o plantio é a cultura da cana já suca que vai ser muito importante e um ou

## Generate mock


In [72]:
chain = get_chain_mock(model="groq")

In [73]:
response_simulado = chain.invoke({"user_query": user_query})

In [74]:
from pydantic import BaseModel, Field
from typing import List

system_prompt = """
Você é um bot chamado Edu e foi desenvolvido pela Nero.AI. Você vai pegar o simulado gerado e criar questões para compor um simulado do ENEM.
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "Simulado: \n\n {mock}")
        ]
)

model_name = "gpt-4o-2024-08-06"

llm = ChatOpenAI(
    model="gpt-4o-2024-08-06", # 100% json output
    temperature=0,
)

class GetMockSchema(BaseModel):
    """Extrai schema de simulados -> questões e gabarito"""
    
    questions: List[str] = Field(description="As 5 principais questões do simulado")
    options: List[str] = Field(description="As 5 alternativas de cada questão")
    answers: List[str] = Field(description="As respostas corretas de cada questão")
    
def format_output_mock(response_schema):
    try:
        args = response_schema.tool_calls[0]['args']
    except:
        return {
            "question": 'EMPTY',
            "options": ['EMPTY'] * 5,
            "answer": 'EMPTY'
        }

    new_output = []
    for q, o, a in zip(args['questions'], args['options'], args['answers']):
        element = {
            "question": q,
            "options": o.split('\n'),
            "answer": a
        }
        new_output.append(element)
    return new_output

llm_with_tools_extraction = llm.bind_tools([GetMockSchema]) #, strict=True)
chain_structured_extraction = prompt | llm_with_tools_extraction | RunnableLambda(format_output_mock)

In [75]:
response_schema = chain_structured_extraction.invoke({"mock": response_simulado})

In [76]:
response_schema

[{'question': 'Qual foi o principal motivo para a colonização portuguesa no Brasil durante o século XVI?',
  'options': ['a) A busca por ouro e prata',
   'b) A expansão da fé católica',
   'c) A exploração da cana-de-açúcar',
   'd) A competição com os holandeses e franceses',
   'e) A necessidade de estabelecer uma rota comercial com a Ásia'],
  'answer': 'c) A exploração da cana-de-açúcar'},
 {'question': 'Como a colonização portuguesa afetou a população indígena no Brasil?',
  'options': ['a) Eles foram integrados à sociedade portuguesa como iguais',
   'b) Eles foram forçados a trabalhar em fazendas e minas',
   'c) Eles foram expulsos de suas terras e forçados a se mudar para outras regiões',
   'd) Eles foram protegidos pelos portugueses e mantiveram sua cultura intacta',
   'e) Eles foram exterminados pela doença e pela violência'],
  'answer': 'b) Eles foram forçados a trabalhar em fazendas e minas'},
 {'question': 'Qual foi o papel dos africanos escravizados na economia colon

In [86]:
def format_output_mock(response_schema):
    try:
        args = response_schema.tool_calls[0]['args']
    except:
        return {
            "question": 'EMPTY',
            "options": ['EMPTY'] * 5,
            "answer": 'EMPTY'
        }

    new_output = []
    for q, o, a in zip(args['questions'], args['options'], args['answers']):
        element = {
            "question": q,
            "options": o.split('\n'),
            "answer": a
        }
        new_output.append(element)
    return new_output

def get_chain_format_schema():
    
    system_prompt = """
    Você é um bot chamado Edu e foi desenvolvido pela Nero.AI. Você vai pegar o simulado gerado e criar questões para compor um simulado do ENEM.
    """

    prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt), 
                ("human", "Simulado: \n\n {mock}")
            ]
    )

    model_name = "gpt-4o-2024-08-06"

    llm = ChatOpenAI(
        model="gpt-4o-2024-08-06", # 100% json output
        temperature=0,
    )

    class GetMockSchema(BaseModel):
        """Extrai schema de simulados -> questões e gabarito"""

        questions: List[str] = Field(description="As 5 principais questões do simulado")
        options: List[str] = Field(description="As 5 alternativas de cada questão")
        answers: List[str] = Field(description="As respostas corretas de cada questão")

    llm_with_tools_extraction = llm.bind_tools([GetMockSchema]) #, strict=True)
    chain_structured_extraction = prompt | llm_with_tools_extraction | RunnableLambda(format_output_mock)
    
    return chain_structured_extraction

def get_chain_mock(retriever_simu, 
                   retriever, 
                   model="openai"):
    
    if model == "openai":
        _chat_ = ChatOpenAI
        _model_name_ = "gpt-4o-2024-08-06"
    elif model == "groq":
        _chat_ = ChatGroq
        _model_name_ = "llama-3.2-90b-text-preview"
    
    template = """
    Você é um assistente de modelo de linguagem de IA que irá gerar simulados. Sua tarefa é separar cada uma das matérias ou temas
    que o usuário deseja estudar e gerar uma curta query descrevendo o temas, para facilitar a busca por similaridade.
    A entrada do usuário pode ter uma ou mais matérias ou temas.
    Forneça esses temas/matérias separados por novas linhas, no formato:
    - Tema1: Descricao do tema1
    - Tema2: Descricao do tema2
    e assim por diante.
    Separe os temas em: {user_query}"""
    
    template_1 = [
        ('system', """
                    Você é um bot chamado Edu e foi desenvolvido pela Nero.AI. 
                    Você irá gerar questões para compor um simulado do ENEM."""),
        ('system', "Use essas questões como exemplo de estrutura de questão\n\n{context_query_simu}\n\n"),
        ('system', "Use esses documentos para embasar o conteúdo das questões\n\n{context_query_video}\n\n"),
        ('system', "Não crie questões unicamente objetivas, como 'O que é?', 'Quem foi?', contextualize breventemente o tema e crie questões que exijam raciocínio."),
        ('system', """Forneça essas perguntas alternativas separadas por novas linhas. \n:
            Siga a estrutura usando markdown:
            Tema: (tema_questao)\n
         
            QUESTÃO 1: (questao_1)\n
            a) alternativa 1
            b) alternativa 2
            c) alternativa 3
            d) alternativa 4
            e) alternativa 5

            QUESTÃO 2: (questao_2)
            a) alternativa 1
            b) alternativa 2
            c) alternativa 3
            d) alternativa 4
            e) alternativa 5

        e assim por diante.
            Gabarito: 1 - (letra correspondente a alternativa)\n
                      2 - (letra correspondente a alternativa)\n
        e assim por diante.
        """),
        ('system', "Gere apenas 5 questões sobre o tema: {tema_questao}"),
    ]
    
    prompt1 = ChatPromptTemplate.from_messages(template_1)
    
    prompt = ChatPromptTemplate.from_template(template)

    llm = _chat_(temperature=0.1, model=_model_name_)

    generate_temas = (
        prompt
        | llm
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    )

    chain_rag_simu = retriever_simu | format_docs
    chain_rag_video = retriever | format_docs

    chain_rag = (
        {
            "context_query_simu": RunnablePassthrough() | chain_rag_simu,
            "context_query_video": RunnablePassthrough() | chain_rag_video,
            "tema_questao": RunnablePassthrough()
        }
        | prompt1
        | llm
        | StrOutputParser()
    )
    
    chain = generate_temas | chain_rag.map() | format_questoes | StrOutputParser()
    
    return chain

In [78]:
chain_structured_extraction = get_chain_format_schema()

In [79]:
chain_structured_extraction.invoke({"mock": response_simulado})

[{'question': 'Qual foi o principal motivo para a colonização portuguesa no Brasil durante o século XVI?',
  'options': ['a) A busca por ouro e prata',
   'b) A expansão da fé católica',
   'c) A exploração da cana-de-açúcar',
   'd) A competição com os holandeses e franceses',
   'e) A necessidade de estabelecer uma rota comercial com a Ásia'],
  'answer': 'c) A exploração da cana-de-açúcar'},
 {'question': 'Como a colonização portuguesa afetou a população indígena no Brasil?',
  'options': ['a) Eles foram integrados à sociedade portuguesa como iguais',
   'b) Eles foram forçados a trabalhar em fazendas e minas',
   'c) Eles foram expulsos de suas terras e forçados a se mudar para outras regiões',
   'd) Eles foram protegidos pelos portugueses e mantiveram sua cultura intacta',
   'e) Eles foram exterminados pela doença e pela violência'],
  'answer': 'b) Eles foram forçados a trabalhar em fazendas e minas'},
 {'question': 'Qual foi o papel dos africanos escravizados na economia colon

## Put together

In [87]:
chain_mock = get_chain_mock(model="groq", retriever_simu=retriever_simu, retriever=retriever)
chain_schema_mock = get_chain_format_schema()

In [88]:
chain_final = chain_mock | chain_schema_mock

In [89]:
response_structured = chain_final.invoke({"user_query": user_query})

In [90]:
response_structured

[{'question': 'Qual foi o principal motivo para a colonização portuguesa no Brasil durante o século XVI?',
  'options': ['a) A busca por ouro e prata',
   'b) A expansão do cristianismo',
   'c) A exploração da cana-de-açúcar',
   'd) A competição com os holandeses e franceses',
   'e) A necessidade de estabelecer uma rota comercial com a Ásia'],
  'answer': 'c) A exploração da cana-de-açúcar'},
 {'question': 'Como a colonização portuguesa afetou a população indígena no Brasil?',
  'options': ['a) A população indígena foi completamente exterminada',
   'b) A população indígena foi integrada à sociedade portuguesa',
   'c) A população indígena foi forçada a trabalhar em condições de escravidão',
   'd) A população indígena foi expulsa do território brasileiro',
   'e) A população indígena foi protegida pelos portugueses'],
  'answer': 'c) A população indígena foi forçada a trabalhar em condições de escravidão'},
 {'question': 'Qual foi o papel da cana-de-açúcar na economia colonial bras

In [91]:
def get_final_chain(model="groq", retriever_simu=retriever_simu, retriever=retriever):
    chain_mock = get_chain_mock(model="groq", retriever_simu=retriever_simu, retriever=retriever)
    chain_schema_mock = get_chain_format_schema()
    
    chain_final = chain_mock | chain_schema_mock
    return chain_final

In [92]:
chain_final = get_final_chain(model="groq", retriever_simu=retriever_simu, retriever=retriever)

In [93]:
response_structured = chain_final.invoke({"user_query": user_query})
response_structured

[{'question': 'Qual foi o principal motivo para a fundação de cidades importantes no Brasil durante o século XVI?',
  'options': ['a) Expulsar os franceses e estabelecer uma estrutura administrativa.',
   'b) Desenvolver a cultura da cana-de-açúcar e trazer africanos para trabalharem forçadamente.',
   'c) Estabelecer relações comerciais com a África e a Europa.',
   'd) Proteger os indígenas e preservar a cultura local.',
   'e) Desenvolver a indústria e o comércio no Brasil.'],
  'answer': 'a'},
 {'question': 'Qual foi o nome dado ao comércio triangular que envolvia Portugal, África e Brasil durante o século XVI?',
  'options': ['a) Comércio Atlântico',
   'b) Comércio Triangular',
   'c) Comércio Colonial',
   'd) Comércio Internacional',
   'e) Comércio Mercantil'],
  'answer': 'b'},
 {'question': 'Qual foi o impacto da colonização no Brasil durante o século XVI?',
  'options': ['a) Desenvolvimento da indústria e do comércio.',
   'b) Proteção dos indígenas e preservação da cultura